# **Gurobi 6: Ejercicio adicional**  

## **Introducción**  
En esta práctica, aprenderás a interpretar los resultados de un modelo de optimización mediante **análisis de sensibilidad** en un **problema de asignación de personal**.  
Se analizarán:  

📌 **Análisis de Sensibilidad en Gurobi:**  
✅ **Costos Reducidos** (Reduced Costs)  
✅ **Precios Duales** (Shadow Prices)  
✅ **Rango de Optimalidad** (Rango de los coeficientes de la función objetivo)  
✅ **Rango de Factibilidad** (Rango de las restricciones)  

---  
    


## **Descripción del Problema: Asignación Óptima de Personal a Proyectos** 👩‍💼👨‍💻  

Una empresa tiene **3 empleados** y **3 proyectos** en los que deben trabajar.  
Cada empleado tiene un **costo por hora** y una **eficiencia en cada proyecto** (expresada en unidades de trabajo por hora).  
Cada proyecto requiere **un mínimo de unidades de trabajo** para completarse en un período dado.  

📌 **Decisiones:**  
- Cuántas horas asignar de cada empleado a cada proyecto (**variable continua** $ x_{e,p} $).  

📌 **Costos y eficiencia:**  

| Empleado | Costo por hora (\$) | Eficiencia en Proyecto 1 | Eficiencia en Proyecto 2 | Eficiencia en Proyecto 3 |
|----------|-----------------|-----------------|-----------------|-----------------|
| A        | 20              | 5               | 6               | 4               |
| B        | 25              | 4               | 7               | 3               |
| C        | 18              | 6               | 5               | 5               |

📌 **Demanda mínima por proyecto:**  

| Proyecto | Unidades de Trabajo Requeridas |
|----------|-------------------------------|
| 1        | 50                            |
| 2        | 60                            |
| 3        | 40                            |

📌 **Función Objetivo:**  
Minimizar el costo total de asignación:  

$$
\min Z = \sum_{e=1}^{n} \sum_{p=1}^{m} C_{e} x_{e,p}
$$  

Donde:  
- $ x_{e,p} $ es la cantidad de horas que el empleado $e$ trabajará en el proyecto $p $.  
- $ C_e$ es el costo por hora del empleado $e $.  

📌 **Restricciones:**  
1. **Cumplimiento de la demanda mínima por proyecto:**  
   - La suma del trabajo efectivo de los empleados en cada proyecto debe ser al menos la cantidad requerida.  
2. **No negatividad:** $ x_{e,p} \geq 0 $  

---  
    

## **Paso 1: Definir Parámetros del Modelo**

In [ ]:
# Descargar e instalar Gurobi desde la fuente oficial
!add-apt-repository -y ppa:deadsnakes/ppa
!apt-get update
!apt-get install -y python3.10
!pip install -i https://pypi.gurobi.com gurobipy

In [ ]:
!pip install gurobipy

# Importar la biblioteca
import gurobipy as gp
from gurobipy import GRB

print("Gurobi instalado y listo para usarse.")

In [ ]:

# Importar Gurobi
import gurobipy as gp
from gurobipy import GRB

# Conjuntos
empleados = ["A", "B", "C"]
proyectos = [1, 2, 3]

# Parámetros
costo_hora = {"A": 20, "B": 25, "C": 18}
eficiencia = {
    "A": {1: 5, 2: 6, 3: 4},
    "B": {1: 4, 2: 7, 3: 3},
    "C": {1: 6, 2: 5, 3: 5}
}
demanda = {1: 50, 2: 60, 3: 40}


## **Paso 2: Definir el Modelo de Optimización en Gurobi**

In [ ]:

# Crear el modelo
modelo = gp.Model("Asignacion_Empleados")

# Variables de decisión (horas asignadas)
x = modelo.addVars(empleados, proyectos, vtype=GRB.CONTINUOUS, lb=0, name="Horas")

# Definir la función objetivo (minimizar costo total)
modelo.setObjective(
    gp.quicksum(costo_hora[e] * x[e, p] for e in empleados for p in proyectos),
    GRB.MINIMIZE
)

# Restricción: Cumplimiento de la demanda mínima por proyecto
for p in proyectos:
    modelo.addConstr(
        gp.quicksum(eficiencia[e][p] * x[e, p] for e in empleados) >= demanda[p],
        name=f"Demanda_Proyecto_{p}"
    )

# Resolver el modelo
modelo.optimize()


## **Paso 3: Análisis de Sensibilidad**

In [ ]:
############BORRAR, DEJAR QUE EL ALUMNO LO DESARROLLE ##########################
# Costos Reducidos (Reduced Costs)
print("\n📊 **Costos Reducidos**")
for v in modelo.getVars():
    print(f"{v.varName}: {v.RC:.2f}")

# Precios Duales (Shadow Prices)
print("\n📊 **Precios Duales (Shadow Prices)**")
for c in modelo.getConstrs():
    print(f"{c.ConstrName}: {c.Pi:.2f}")

# Rango de Optimalidad (Coeficientes de la función objetivo)
print("\n📊 **Rango de Optimalidad**")
for v in modelo.getVars():
    print(f"{v.varName}: {v.SAObjLow:.2f} <= coef <= {v.SAObjUp:.2f}")

# Rango de Factibilidad (Lado derecho de las restricciones)
print("\n📊 **Rango de Factibilidad**")
for c in modelo.getConstrs():
    print(f"{c.ConstrName}: {c.SARHSLow:.2f} <= RHS <= {c.SARHSUp:.2f}")

################################################################################



## **📌 Interpretación de los Resultados**  

✅ **Costos Reducidos:** Indican cuánto debería cambiar el coeficiente de la función objetivo para que la variable entre en la solución óptima.  
✅ **Precios Duales:** Muestran cuánto aumentaría el valor óptimo si se incrementara la demanda mínima en un proyecto en una unidad.  
✅ **Rango de Optimalidad:** Indica el intervalo dentro del cual un coeficiente de la función objetivo puede variar sin cambiar la solución óptima.  
✅ **Rango de Factibilidad:** Indica el intervalo dentro del cual la demanda mínima de un proyecto puede cambiar sin afectar la base óptima.  

---  

## **Ejercicios para el Alumno**  

1. **Modifica los costos por hora y analiza los cambios en los rangos de optimalidad.**  
2. **Aumenta la demanda del Proyecto 2 en 10 unidades y analiza el nuevo precio dual.**  
3. **Cambia la eficiencia del Empleado B en el Proyecto 1 y observa su impacto en los costos reducidos.**  
4. **Interpreta los resultados y explica en qué situaciones la empresa podría beneficiarse de cambios en la asignación de personal.**  

✍️ **Instrucciones:**  
- Modifica los parámetros en el código y vuelve a ejecutar el modelo.  
- Analiza los resultados e identifica los cambios en los valores de sensibilidad.  

🚀 ¡Ahora pon en práctica el análisis de sensibilidad con Gurobi!  
    